# Decentralized Crowd Density (DCD) Maps

First draft of DCD analysis.

## Export structure from OMNeT++

* Each node creates a single file with the map state for each time step
* Global DCD module creates a separate file for the groud 
  truth of the simulation
* todo: write all *node-data* into single file.

### Node map file structure

* only not null values
* first line with `#` contains meta data used for processing in python
* NODE_ID=0a:aa:00:00:00:02 (mac address)
* compound key: [simtime, x, y]
* values:
  * _count_: number of nodes in given cell
  * _measured_t_:   time when count was measured (does not need to be the given node)
  * _received_t_:   time when that measurement was received. If the given node measured the 
                    value itself `received_t` and `simtime` are equal!
  * _source_:       NODE_ID which created the measurement
  * _own_cell_:     If `1` the cell with [x,y] of this row is the current location of the node.
                    Example. node_0a:aa:00:00:00:02 is in cell [66,75] at time 2.0 sec

```
#CELLSIZE=3.000000,DATACOL=-1,IDXCOL=3,NODE_ID=0a:aa:00:00:00:02,SEP=;,XSIZE=281.135000,YSIZE=233.492000
simtime;x;y;count;measured_t;received_t;source;own_cell
2;5;46;1;2;2;0a:aa:00:00:00:02;0
2;24;73;1;2;2;0a:aa:00:00:00:02;0
2;25;73;1;2;2;0a:aa:00:00:00:02;0
2;66;75;1;2;2;0a:aa:00:00:00:02;1
4;5;46;1;4;4;0a:aa:00:00:00:02;0
```

### Global map file structure

* same meta data in first line starting with `#`
* values:
  * same as node map file
  * _node_id_:  String list of node_id's contained in the given cell [x,y]. The list is separated by `,` and 
                not by `;` as indicated by `SEP` in the metadata. This column will be normalized later 
                during python processing. 

```
#CELLSIZE=3.000000,DATACOL=-1,IDXCOL=3,SEP=;,NODE_ID=global,XSIZE=281.135000,YSIZE=233.492000
simtime;x;y;count;measured_t;received_t;source;own_cell;node_id
2;4;46;1;2;2;global;0;0a:aa:00:00:00:08
2;5;46;1;2;2;global;0;0a:aa:00:00:00:04
2;23;72;1;2;2;global;0;0a:aa:00:00:00:03
2;24;73;1;2;2;global;0;0a:aa:00:00:00:06
...
4;5;47;2;4;4;global;0;0a:aa:00:00:00:04,0a:aa:00:00:00:08
```

## First analysis

Class structure and placement in crownetutils is not fixed yet.

In [ ]:
%matplotlib widget

In [ ]:
import os
import sys
import matplotlib.pyplot as plt

# if file part of repo add repo to import path
curr_path = os.path.abspath('.')
if curr_path.endswith("crownetutils/tutorials"):
    sys.path.append(os.path.split(curr_path)[0])

import pandas as pd

from crownetutils.oppanalyzer.dcd import (
    DcdMap2D, DcdMap2DMulti
)

from crownetutils.uitls.path import PathHelper
from crownetutils.vadereanalyzer.plots.scenario import VaderScenarioPlotHelper


In [ ]:
# base paths for example

simulation = "mucFreiNetdLTE2dMulticast"
# run = "0_vadere00_geo_20201026-ymf_map"
run = "0_vadere00_geo_20201103-10:32:09_all"

p = PathHelper.rover_sim(simulation, run)

scenario_path = p.glob("vadere.d/*.scenario", recursive=False, expect=1)
node_paths = p.glob("0a:*.csv")
global_path = p.abs_path("global.csv")

* create `VadereScenarioPlotHelper` to add obstacles to map plots. 
* Read csv files into simple pandas.DataFrames (set multiIndex)
  * real_coord=True --> translates cell ids such as [3,5] to the correct values. (e.g with cell_size=3.0 --> [9.0, 15.0]
  * full_map=False  --> do not create missing cells. They will be created lated if needed.

In [ ]:
s_plotter =  VaderScenarioPlotHelper(scenario_path)

### DcdMap2D

Simple class to combine an manipulate DCD map data:

* replace NODE_IDs with integers for easier indexing and slicing
* _delay_:    `received_t - measured_t`
* _measurement_age_: `simtime - measured_t`
* _update_age_: `simtime - received_t`

In [ ]:
# dcd = DcdMap2D.from_paths(global_path, node_paths)
dcd = DcdMap2DMulti.from_paths(global_path, node_paths)
dcd.set_scenario_plotter(s_plotter)

# plot_wrapper to ensure smooth handling for Jupyter notebook.
# default plot_wrapper does nothing!
def plot_wrapper(func, _self, *args, **kwargs):
    # ensure plot is called with some existing Axes. If not create figure with one axis.
    if "ax" not in kwargs:
        print("create axes...")
        fig, ax = plt.subplots(1, 1, figsize=(10, 7))
        fig.canvas.toolbar_position = 'bottom'
        kwargs.setdefault("ax", ax)
    
    # execute actual plot function
    ret = func(_self, *args, **kwargs)
    return ret

dcd.plot_wrapper = plot_wrapper

        

#main pd.DataFrame
dcd.raw2d.head(5)

* descriptive stats over all nodes and cells

In [ ]:
dcd.describe_raw()

* descriptive stats only for global map

In [ ]:
dcd.describe_raw(global_only=True)

## Simple Plots

todo: synchronize color coding!


### Node 3(...:04) Plots

* plot1: Map seen from Node 3(...:04) GREEN  at [15.0, 141.0] [same place as pink 7(...:08)]
* plot2: Node placement taken form global state (ground truth)
* plot3: total count of nodes per simtime. 
  * Blue line is ground truth (7 Nodes)
  * Red Node 3(...:04) is the current node.
  * 'Old' measures are counted. This is the reason for the growing number of nodes.


In [ ]:
p1 = dcd.plot_summary(4.0, 3, "(data from all 'ages')")

In [ ]:
p = dcd.plot_density_map(12, 5, make_interactive=True
)

### Node 3(...:04) Plots at 12.0 second
* density shows 'path'
* look at Node 6(...:07) brown 
  * Node 3 has one measurment [84.0, 84.0],
  * and 3 additional measurment form Node 2(...:03) orange.
   

In [ ]:
#p2 = dcd.plot2(12.0, 3, "(data from all 'ages')")
p2 = dcd.plot_count()

In [ ]:
pd.set_option('display.max_rows', 1000)
idx = pd.IndexSlice
node_id = 2
sel = idx[2, 12:18, :, :]
col = ("count", "measured_t", "received_t", "source", "own_cell")

In [ ]:
df = dcd.raw2d.loc[sel, col]
df.groupby("simtime").sum()["count"].to_frame()

In [ ]:
# node 5 is the current node
df.groupby(["simtime", "source"]).sum()["count"].to_frame()

In [ ]:
ret = dcd.plot_density_map_interactive()

In [ ]:
mask = (df["count"] == 0) 
#df.loc[mask]
df

In [ ]:
idx2 = pd.IndexSlice
sel = idx[12:14, :]
dcd.glb_loc_df.loc[sel]